In [6]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense,Dropout
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers,optimizers
import numpy as np
import keras

In [2]:
from keras.datasets import cifar10


In [19]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping
from scipy.misc import toimage, imresize

In [5]:
#Defining the shape of input image
no_of_channels = 3
height = 64
width = 64

In [4]:
#Resizing the dataset
def Resize() :
    X = np.zeros((images.shape[0],height,width,3))
    for i in range(images.shape[0]):
        X[i]= imresize(images[i], (height,width,no_of_channels), interp='bilinear', mode=None)
    return X

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170491904/170498071 [============================>.] - ETA: 0s

In [34]:
np.save("x_train_out.npy",x_train)
np.save("y_train_out.npy",y_train)
np.save("x_test_out.npy",x_test)
np.save("y_test_out.npy",y_test)

In [34]:
#Loading testing and training data from numpy arrays
x_train = np.load("x_train_out.npy")
y_train = np.load("y_train_out.npy")
x_test = np.load("x_test_out.npy")
y_test = np.load("y_test_out.npy")

In [35]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [36]:
x_train.shape

(50000, 32, 32, 3)

In [37]:
#Changing the shape of CIFAR10 dataset from 32*32*3 to 64*64*3
x_train = Resize(x_train,64,64)
x_test = Resize(x_test,64,64)

In [ ]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [ ]:
print(y_train.shape)

In [8]:
print(y_train[0])

[6]


In [9]:
#one hot encoding of target labels
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [10]:
print(y_train.shape)

(50000, 10)


In [11]:
print(y_train[0])

[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


In [10]:
def PretrainedModel(num_of_classes) :
    
    #load vgg model from keras
    vgg_16_model = VGG16(weights=None,include_top= False,input_shape=(height,width,no_of_channels))
    vgg_16_model.summary()
    
    #get weights from pretrained model on imagenet
    vgg_16_model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
    print(len(vgg_16_model.layers))
    
    #Freeze all layers
    #for i in range(19):
     # vgg_16_model.layers[i].trainable = False
    
    #change model input layer according to cifar-10 dataset
    inputs = Input(shape = (height,width,no_of_channels), name = "image_input")
    
    #create dummy layer
    output_vgg16_model = vgg_16_model(inputs)
    
    #Add the fully-connected layers 
    #Adding one fully connected layer instead of 2 to decrease overfitting
    x = Flatten(name='flatten')(output_vgg16_model)
    x = Dense(2048, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_of_classes, activation='softmax', name='predictions')(x)

    #Create custom model
    cifar10_vgg = Model(inputs=inputs, outputs=x)

    #In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
    cifar10_vgg.summary()
    
    print(len(cifar10_vgg.layers))
    
    return cifar10_vgg

    

In [12]:
model = PretrainedModel(10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [13]:
data_augmentation = True

In [14]:
batch_size = 64
epochs = 100

In [18]:
#Defining Callback functions which will be called by model during runtime when specified condition satisfies
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('./vgg16imagenetpretrained_upsampleimage_cifar10_data_argumentation.csv')
early_stopper = EarlyStopping(min_delta=0.001, patience=20)

In [15]:
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [17]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])

In [26]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,verbose=1,validation_data=(x_test,y_test),callbacks=[lr_reducer, early_stopper, csv_logger])
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,callbacks=[lr_reducer, early_stopper, csv_logger])
    
    
    

-------------Using Data augmentation------------
Epoch 1/5
390/390 [==============================] - 4666s - loss: 14.5056 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 2/5
390/390 [==============================] - 4736s - loss: 14.5084 - acc: 0.0999 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 3/5
390/390 [==============================] - 4755s - loss: 14.5062 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 4/5
390/390 [==============================] - 4815s - loss: 14.5033 - acc: 0.1002 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 5/5
390/390 [==============================] - 4829s - loss: 14.5067 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000


In [27]:
model.save_weights("cifar_adam.h5")

In [65]:
model.save_weights("cifar_rms_epoch25.h5")

In [21]:
model.load_weights("cifar_rms_epoch25.h5")

In [22]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 204s   

Test result: 68.600 loss: 1.507
